In [2]:
from graph3d import *

/root/anaconda3/envs/deepseek/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:
!pwd

/home/cz/MolTC-main/model


In [1]:
import sys

# 添加新的路径
sys.path.append('/work/home/ac15y5ara4/cz/MolTC-main/')

In [2]:
"""
 Copyright (c) 2023, salesforce.com, inc.
 All rights reserved.
 SPDX-License-Identifier: BSD-3-Clause
 For full license text, see the LICENSE file in the repo root or https://opensource.org/licenses/BSD-3-Clause
"""
import logging
import torch
import torch.nn as nn
from torch.cuda.amp import autocast as autocast
from torch.nn import functional as F
from peft import get_peft_config, get_peft_model, get_peft_model_state_dict, LoraConfig, TaskType, PeftModel
from ogb.utils import smiles2graph
from torch_geometric.loader.dataloader import Collater
from torch_geometric.data import Data
import numpy as np
from lavis.models.blip2_models.blip2 import (
    # Blip2Base,
    disabled_train,
)
from model.blip2 import Blip2Base
from transformers import AutoTokenizer
from transformers import OPTForCausalLM
# from opendelta import LoraModel
# from opendelta.delta_models.lora import LoraConfig
# from opendelta.delta_configs

opt_model_list = [
    "facebook/galactica-125m",
    "facebook/galactica-1.3b",
    "facebook/galactica-6.7b",
    "facebook/galactica-30b",
]

def mask_by_len(input, lens, fill_value=0):
    '''
    input: shape = [N, D]
    lens: shape = [N]
    '''
    mask = torch.arange(input.shape[1], device=input.device).reshape(1, -1)
    mask = mask < lens.reshape(-1, 1)
    input[mask] = fill_value
    return input


def smiles2data(smiles):
    graph = smiles2graph(smiles)
    x = torch.from_numpy(graph['node_feat'])
    edge_index = torch.from_numpy(graph['edge_index'], )
    edge_attr = torch.from_numpy(graph['edge_feat'])
    data = Data(x=x, edge_index=edge_index, edge_attr=edge_attr)
    return data

import re
SPLIT_MARKER = f"SPL{1}T-TH{1}S-Pl3A5E"

CUSTOM_SEQ_RE = re.compile(r"(\[START_(DNA|SMILES|I_SMILES|AMINO)])(.*?)(\[END_\2])")


def _insert_split_marker(m: re.Match):
    """
    Applies split marker based on a regex match of special tokens such as
    [START_DNA].

    Parameters
    ----------
    n : str
        Input text to split

    Returns
    ----------
    str - the text with the split token added
    """
    start_token, _, sequence, end_token = m.groups()
    sequence = re.sub(r"(.)", fr"{SPLIT_MARKER}\1", sequence, flags=re.DOTALL)
    return f"{start_token}{sequence}{SPLIT_MARKER}{end_token}"

def escape_custom_split_sequence(text):
    """
    Applies custom splitting to the text for GALILEO's tokenization

    Parameters
    ----------
    text : str
        Input text to split

    Returns
    ----------
    str - the text with the split token added
    """
    return CUSTOM_SEQ_RE.sub(_insert_split_marker, text)

def smiles_handler(text, mol_ph):
    smiles_list = []
    for match in CUSTOM_SEQ_RE.finditer(text):
        smiles = match.group(3)
        smiles_list.append(smiles)
    
    text = CUSTOM_SEQ_RE.sub(r'\1\3\4%s' % (mol_ph), text)
    text = escape_custom_split_sequence(text)
    return text, smiles_list

/opt/conda/envs/deepseek/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/opt/conda/envs/deepseek/lib/python3.10/site-packages/fairscale/experimental/nn/offload.py:19: FutureWarning: `torch.cuda.amp.custom_fwd(args...)` is deprecated. Please use `torch.amp.custom_fwd(args..., device_type='cuda')` instead.
  return torch.cuda.amp.custom_fwd(orig_func)  # type: ignore
/opt/conda/envs/deepseek/lib/python3.10/site-packages/fairscale/experimental/nn/offload.py:30: FutureWarning: `torch.cuda.amp.custom_bwd(args...)` is deprecated. Please use `torch.amp.custom_bwd(args..., device_type='cuda')` instead.
  return torch.cuda.amp.custom_bwd(orig_func)  # type: ignore
2025-03-17 10:07:21.542503: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register f

fused_multi_tensor is not installed corrected
fused_rounding is not installed corrected
fused_layer_norm is not installed corrected
fused_softmax is not installed corrected


In [3]:
from easydict import EasyDict

args = EasyDict({
    "filename": "ft_ddi_value_stage2_new16",
    "seed": 42,
    "mode": "ft",
    "strategy_name": None,
    "iupac_prediction": False,
    "ckpt_path": None,
    "gin_hidden_dim": 300,
    "gin_num_layers": 5,
    "drop_ratio": 0.0,
    "tune_gnn": True,
    "bert_hidden_dim": 768,
    "bert_name": "scibert",
    "cross_attention_freq": 2,
    "num_query_token": 8,
    "opt_model": "facebook/galactica-1.3b",
    "num_beams": 5,
    "do_sample": False,
    "max_len": 30,
    "min_len": 8,
    "llm_tune": "lora",
    "peft_config": None,
    "peft_dir": "",
    "save_every_n_epochs": 10,
    "load_in_8bit": False,
    "lora_r": 8,
    "lora_alpha": 32,
    "lora_dropout": 0.1,
    "reaction_weight": 1.0,
    "weight_decay": 0.05,
    "init_lr": 0.0001,
    "min_lr": 1.0e-05,
    "warmup_lr": 1.0e-06,
    "warmup_steps": 1000,
    "lr_decay_rate": 0.9,
    "scheduler": "linear_warmup_cosine_lr",
    "stage1_path": "",
    "stage2_path": "all_checkpoints/stage2/last.ckpt",
    "init_checkpoint": "all_checkpoints/pretrain1/last.ckpt",
    "caption_eval_epoch": 50,
    "num_workers": 2,
    "batch_size": 20,
    "inference_batch_size": 4,
    "use_smiles": False,
    "root": "/home/cz/MolTC-main/data/ddi_data/Zhangddi_data/train/",
    "text_max_len": 128,
    "prompt": "[START_I_SMILES]{}[END_I_SMILES]. ",
    "accelerator": "gpu",
    "devices": "0,1",
    "precision": "bf16",
    "max_epochs": 1000,
    "accumulate_grad_batches": 1,
    "check_val_every_n_epoch": 1,
    "double": True,
    "solve": False,
    "valid_root": "/home/cz/MolTC-main/data/ddi_data/Zhangddi_data/valid/",
    "DDI": True,
    "fangguangtuan": False,
    "zijiegou":False
})


In [4]:
class Blip2OPT(Blip2Base):
    """
    BLIP2 first-stage model with Q-former and ViT.
    Supported model types:
        - pretrained: pretrained model with vit-g
        - pretrain_vitL: pretrained model with vit-large
        - coco: fintuned model on coco
    Usage:
        >>> from lavis.models import load_model
        >>> model = load_model("blip2", "pretrain")
    """
    def __init__(
        self,
        bert_name,
        gin_num_layers,
        gin_hidden_dim,
        gin_drop_ratio,
        tune_gnn=False,
        num_query_token=32,
        cross_attention_freq=2,
        llm_tune='freeze',
        peft_dir='',
        opt_model="facebook/galactica-1.3b",
        prompt="",
        args=None,
    ):
        super().__init__()
        self.args = args

        self.graph_encoder, self.ln_graph = self.init_graph_encoder(gin_num_layers, gin_hidden_dim, gin_drop_ratio)
        self.tune_gnn = tune_gnn
        if not tune_gnn:
            for name, param in self.graph_encoder.named_parameters():
                param.requires_grad = False
            self.graph_encoder = self.graph_encoder.eval()
            self.graph_encoder.train = disabled_train
            logging.info("freeze graph encoder")
        
        self.num_query_token = num_query_token
        self.Qformer, self.query_tokens = self.init_Qformer(bert_name, num_query_token, self.graph_encoder.num_features, cross_attention_freq)
        ### remove the unused parameters
        self.Qformer.cls = None
        self.Qformer.bert.embeddings.word_embeddings = None
        self.Qformer.bert.embeddings.position_embeddings = None
        for layer in self.Qformer.bert.encoder.layer:
            layer.output = None
            layer.intermediate = None
        #####################################
        ## initialize opt model

        # opt_model = "./galactica-1.3b"
        opt_model = "/work/home/ac15y5ara4/cz/MolTC-main/deepseek/falcon/7b"
        self.opt_tokenizer = AutoTokenizer.from_pretrained(opt_model, use_fast=False, padding_side='right')
        self.opt_tokenizer.add_special_tokens({'pad_token': '<pad>', 'sep_token': '</s>'})
        self.opt_tokenizer.add_tokens('<mol>') # molecule placeholder
        self.mol_token = '<mol>'
        self.opt_tokenizer.mol_token_id = self.opt_tokenizer("<mol>", add_special_tokens=False).input_ids[0]

        self.collater = Collater([], [])
        
        if opt_model == 'facebook/galactica-125m':
            self.opt_model = OPTForCausalLM.from_pretrained(opt_model)
        else:
            # self.opt_model = OPTForCausalLM.from_pretrained(opt_model, torch_dtype=torch.float16)
            self.opt_model = OPTForCausalLM.from_pretrained(opt_model, torch_dtype=torch.bfloat16)
        self.opt_model.resize_token_embeddings(len(self.opt_tokenizer)) ## this will cause bug when full fine-tuning the opt model

        self.llm_tune = llm_tune
        if llm_tune == 'lora':
            if peft_dir:
                self.opt_model = PeftModel.from_pretrained(self.opt_model, peft_dir, is_trainable=True)
                #for name, param in self.opt_model.named_parameters():
                #    param.requires_grad = False
            else:
                if self.args.peft_config:
                    peft_config = LoraConfig(**LoraConfig.from_json_file(self.args.peft_config))
                else:
                    peft_config = LoraConfig(task_type=TaskType.CAUSAL_LM, inference_mode=False, r=args.lora_r, lora_alpha=args.lora_alpha, lora_dropout=args.lora_dropout)
                self.peft_config = peft_config
                self.opt_model = get_peft_model(self.opt_model, peft_config)
            self.opt_model.print_trainable_parameters()
        elif llm_tune == 'freeze':
            for name, param in self.opt_model.named_parameters():
                param.requires_grad = False
        elif llm_tune == 'full':
            pass
        else:
            raise NotImplementedError()

        ## fixme: this is different from the original BLIP2
        self.eos_token_id = self.opt_tokenizer(
            "\n", add_special_tokens=False
        ).input_ids[0]

        self.opt_proj = nn.Linear(
            self.Qformer.config.hidden_size, self.opt_model.config.hidden_size
        )
        
        ## fixme: no prompt yet
        self.prompt = prompt
        # prompt_tokens = self.opt_tokenizer(self.prompt, return_tensors="pt")
        # self.prompt_length = prompt_tokens.attention_mask.sum(1)

    def forward_old(self, batch):
        graphs, text_tokens, prompt_lens = batch
        graph_embeds, graph_masks = self.graph_encoder(graphs)
        if not self.tune_gnn:
            graph_embeds = graph_embeds.detach()
        graph_embeds = self.ln_graph(graph_embeds, graph_masks)
        device = graph_embeds.device
        query_tokens = self.query_tokens.expand(graph_embeds.shape[0], -1, -1)
        query_output = self.Qformer.bert(
            query_embeds=query_tokens,
            encoder_hidden_states=graph_embeds,
            encoder_attention_mask=graph_masks, # fixme: check whether this mask is correct
            return_dict=True,
        )
        inputs_opt = self.opt_proj(query_output.last_hidden_state)
        atts_opt = torch.ones(inputs_opt.size()[:-1], dtype=torch.long).to(device)
        targets = text_tokens.input_ids.masked_fill(
            text_tokens.input_ids == self.opt_tokenizer.pad_token_id, -100
        )
        if self.prompt:
            targets = mask_by_len(targets, prompt_lens, -100) # do not apply loss to the prompt
            # targets[:, : self.prompt_length] = -100  # do not apply loss to the prompt
        
        empty_targets = (
            torch.ones(atts_opt.size(), dtype=torch.long).to(device).fill_(-100)
        )
        targets = torch.cat([empty_targets, targets], dim=1)
        
        inputs_embeds = self.opt_model.get_input_embeddings()(text_tokens.input_ids)
        inputs_embeds = torch.cat([inputs_opt, inputs_embeds], dim=1)
        attention_mask = torch.cat([atts_opt, text_tokens.attention_mask], dim=1)
        
        outputs = self.opt_model(
            inputs_embeds=inputs_embeds,
            attention_mask=attention_mask,
            return_dict=True,
            labels=targets,
        )
        loss = outputs.loss
        return {"loss": loss}
    
    def forward(self, batch):
        if self.args.double==False and self.args.solve == False and self.args.fangguangtuan == False and self.args.DDI == False:
            graphs, prompt_tokens, text_tokens = batch
            #print(graphs)pyg图数据格式
            #print(prompt_tokens)下面两个都是64 54 38这种
            #print(text_tokens)
            graph_embeds, graph_masks = self.graph_encoder(graphs)
            if not self.tune_gnn:
                graph_embeds = graph_embeds.detach()
            graph_embeds = self.ln_graph(graph_embeds, graph_masks)
            #print(graph_embeds)应该是读出完之后的向量了
            device = graph_embeds.device
            query_tokens = self.query_tokens.expand(graph_embeds.shape[0], -1, -1)
            #print(query_tokens)也是向量罢了
            query_output = self.Qformer.bert(
                query_embeds=query_tokens,
                encoder_hidden_states=graph_embeds,
                encoder_attention_mask=graph_masks, # fixme: check whether this mask is correct
                return_dict=True,
            )
            mol_tokens = self.opt_proj(query_output.last_hidden_state)
            #print(mol_tokens)
            #这里应该是去取出了分子的向量,并不是token
            empty_targets = torch.ones(prompt_tokens.attention_mask.shape, dtype=torch.long).to(device).fill_(-100)
            targets = text_tokens.input_ids.masked_fill(
                text_tokens.input_ids == self.opt_tokenizer.pad_token_id, -100
            )
            targets = torch.cat([empty_targets, targets], dim=1)
            prompt_embeds = self.opt_model.get_input_embeddings()(prompt_tokens.input_ids)
            prompt_embeds[prompt_tokens.is_mol_token] = mol_tokens.flatten(0, 1)
            inputs_embeds = self.opt_model.get_input_embeddings()(text_tokens.input_ids)
            inputs_embeds = torch.cat((prompt_embeds, inputs_embeds), dim=1)
            attention_mask = torch.cat([prompt_tokens.attention_mask, text_tokens.attention_mask], dim=1)
            #print(targets)
            outputs = self.opt_model(
                inputs_embeds=inputs_embeds,
                attention_mask=attention_mask,
                return_dict=True,
                labels=targets,
            ) # type: ignore
            loss = outputs.loss
            return {"loss": loss}
        else:
            graphs1, graphs2,prompt_tokens, text_tokens= batch
            #print(graphs)pyg图数据格式
            #print(prompt_tokens)下面两个都是64 54 38这种
            #print(text_tokens)
            graph_embeds1, graph_masks1 = self.graph_encoder(graphs1)
            if not self.tune_gnn:
                graph_embeds1 = graph_embeds1.detach()
            graph_embeds1 = self.ln_graph(graph_embeds1, graph_masks1)
            #print(graph_embeds)应该是读出完之后的向量了
            device = graph_embeds1.device
            query_tokens1 = self.query_tokens.expand(graph_embeds1.shape[0], -1, -1)
            #print(query_tokens1.size())#也是向量罢了
            query_output1 = self.Qformer.bert(
                query_embeds=query_tokens1,
                encoder_hidden_states=graph_embeds1,
                encoder_attention_mask=graph_masks1, # fixme: check whether this mask is correct
                return_dict=True,
            )
            #print(query_output1.size())
            mol_tokens1 = self.opt_proj(query_output1.last_hidden_state)
            #print(mol_tokens1.size())
            #这里应该是去取出了分子的向量,并不是token，这里是将768层 变换称了2048层
            
            
            
            graph_embeds2, graph_masks2 = self.graph_encoder(graphs2)
            if not self.tune_gnn:
                graph_embeds2 = graph_embeds1.detach()
            graph_embeds2 = self.ln_graph(graph_embeds2, graph_masks2)
            #print(graph_embeds)应该是读出完之后的向量了
            device = graph_embeds2.device
            query_tokens2 = self.query_tokens.expand(graph_embeds2.shape[0], -1, -1)
            #print(query_tokens)也是向量罢了
            query_output2 = self.Qformer.bert(
                query_embeds=query_tokens2,
                encoder_hidden_states=graph_embeds2,
                encoder_attention_mask=graph_masks2, # fixme: check whether this mask is correct
                return_dict=True,
            )
            mol_tokens2 = self.opt_proj(query_output2.last_hidden_state)
            
            
            
            
            
            mol_tokens = torch.cat([mol_tokens1,mol_tokens2], dim=1)
            empty_targets = torch.ones(prompt_tokens.attention_mask.shape, dtype=torch.long).to(device).fill_(-100)
            targets = text_tokens.input_ids.masked_fill(
                text_tokens.input_ids == self.opt_tokenizer.pad_token_id, -100
            )
            targets = torch.cat([empty_targets, targets], dim=1)
            prompt_embeds = self.opt_model.get_input_embeddings()(prompt_tokens.input_ids)
            prompt_embeds[prompt_tokens.is_mol_token] = mol_tokens.flatten(0, 1)
            inputs_embeds = self.opt_model.get_input_embeddings()(text_tokens.input_ids)
            inputs_embeds = torch.cat((prompt_embeds, inputs_embeds), dim=1)
            attention_mask = torch.cat([prompt_tokens.attention_mask, text_tokens.attention_mask], dim=1)
            
            
            outputs = self.opt_model(
                inputs_embeds=inputs_embeds,
                attention_mask=attention_mask,
                return_dict=True,
                labels=targets,
            )
            loss = outputs.loss
            return {"loss": loss}

    def forward_reaction(self, batch):
        reaction_tokens, notes_tokens, graphs = batch
        graph_embeds, graph_masks = self.graph_encoder(graphs)
        if not self.tune_gnn:
            graph_embeds = graph_embeds.detach()
        graph_embeds = self.ln_graph(graph_embeds, graph_masks)
        device = graph_embeds.device
        query_tokens = self.query_tokens.expand(graph_embeds.shape[0], -1, -1)
        query_output = self.Qformer.bert(
            query_embeds=query_tokens,
            encoder_hidden_states=graph_embeds,
            encoder_attention_mask=graph_masks, # fixme: check whether this mask is correct
            return_dict=True,
        )
        mol_tokens = self.opt_proj(query_output.last_hidden_state) # shape = [mol_num, num_query_token, D]

        if False:
            if self.llm_tune:
                react_embeds = self.opt_model.model.get_decoder().embed_tokens(reaction_tokens.input_ids) # shape = [B, max_len, D]
                notes_embeds = self.opt_model.model.get_decoder().embed_tokens(notes_tokens.input_ids)
            else:
                react_embeds = self.opt_model.model.decoder.embed_tokens(reaction_tokens.input_ids) # shape = [B, max_len, D]
                notes_embeds = self.opt_model.model.decoder.embed_tokens(notes_tokens.input_ids) # shape = [B, max_len, D]
        else:
            react_embeds = self.opt_model.get_input_embeddings()(reaction_tokens.input_ids)
            notes_embeds = self.opt_model.get_input_embeddings()(notes_tokens.input_ids)

        react_embeds[reaction_tokens.is_ph_token] = mol_tokens.flatten(0, 1)
        inputs_embeds = torch.cat((react_embeds, notes_embeds), dim=1)

        targets = notes_tokens.input_ids.masked_fill(
            notes_tokens.input_ids == self.opt_tokenizer.pad_token_id, -100
        )
        empty_targets = (
            torch.ones(reaction_tokens.attention_mask.shape, dtype=torch.long).to(device).fill_(-100)
        )
        targets = torch.cat([empty_targets, targets], dim=1)
        attention_mask = torch.cat([reaction_tokens.attention_mask, notes_tokens.attention_mask], dim=1)

        outputs = self.opt_model(
            inputs_embeds=inputs_embeds,
            attention_mask=attention_mask,
            return_dict=True,
            labels=targets,
        )
        loss = outputs.loss
        return {"loss": loss}

    @torch.no_grad()
    def generate_old(
        self,
        samples,
        do_sample=False,
        num_beams=5,
        max_length=128,
        min_length=1,
        top_p=0.9,
        repetition_penalty=1.0,
        length_penalty=1.0,
        num_captions=1,
        temperature=1,
    ):
        """
        Args:
            samples (dict): A dictionary containing the following keys:
                - image (torch.Tensor): A tensor of shape (batch_size, 3, H, W)
            num_beams (int): Number of beams for beam search. 1 means no beam search.
            max_length (int): The maximum length of the sequence to be generated.
            min_length (int): The minimum length of the sequence to be generated.
            top_p (float): The cumulative probability for nucleus sampling.
            repetition_penalty (float): The parameter for repetition penalty. 1.0 means no penalty.
            num_captions (int): Number of captions to be generated for each image.
        Returns:
            captions (list): A list of strings of length batch_size * num_captions.
        """
        graphs = samples['graphs']
        prompt_tokens = samples['prompt_tokens']
        # prompt_lens = samples['prompt_lens']
        with self.maybe_autocast():
            graph_embeds, graph_masks = self.graph_encoder(graphs)
            graph_embeds = self.ln_graph(graph_embeds)

            query_tokens = self.query_tokens.expand(graph_embeds.shape[0], -1, -1)
            query_output = self.Qformer.bert(
                query_embeds=query_tokens,
                encoder_hidden_states=graph_embeds,
                encoder_attention_mask=graph_masks,
                return_dict=True,
            )

            device = graph_embeds.device
            inputs_opt = self.opt_proj(query_output.last_hidden_state)
            atts_opt = torch.ones(inputs_opt.size()[:-1], dtype=torch.long, device=device)

            attention_mask = torch.cat([atts_opt, prompt_tokens.attention_mask], dim=1)
            
            inputs_embeds = self.opt_model.get_input_embeddings()(prompt_tokens.input_ids)
            inputs_embeds = torch.cat([inputs_opt, inputs_embeds], dim=1)

            outputs = self.opt_model.generate(
                inputs_embeds=inputs_embeds,
                attention_mask=attention_mask,
                do_sample=do_sample,
                top_p=top_p,
                temperature=temperature,
                num_beams=num_beams,
                max_length=max_length,
                min_length=min_length,
                # pad_token_id=self.pad_token_id,
                eos_token_id=self.eos_token_id,
                repetition_penalty=repetition_penalty,
                length_penalty=length_penalty,
                num_return_sequences=num_captions,
                return_dict_in_generate=True
                # use_cache=False,
            )
            return outputs
            output_text = self.opt_tokenizer.batch_decode(outputs, skip_special_tokens=True)
            
            output_text = [text.strip() for text in output_text]
            return output_text
    
    @torch.no_grad()
    def generate(
        self,
        samples,
        # max_new_tokens=30,
        do_sample=False,
        num_beams=5,
        max_length=128,
        min_length=1,
        top_p=0.9,
        repetition_penalty=1.0,
        length_penalty=1.0,
        num_captions=1,
        temperature=1,
    ):
        """
        Args:
            samples (dict): A dictionary containing the following keys:
                - image (torch.Tensor): A tensor of shape (batch_size, 3, H, W)
            num_beams (int): Number of beams for beam search. 1 means no beam search.
            max_length (int): The maximum length of the sequence to be generated.
            min_length (int): The minimum length of the sequence to be generated.
            top_p (float): The cumulative probability for nucleus sampling.
            repetition_penalty (float): The parameter for repetition penalty. 1.0 means no penalty.
            num_captions (int): Number of captions to be generated for each image.
        Returns:
            captions (list): A list of strings of length batch_size * num_captions.
        """
        print("开始生成")
        if self.args.double == True or self.args.solve == True or self.args.fangguangtuan == True or self.args.DDI == True:
            graphs1 = samples['graphs1']
            prompt_tokens = samples['prompt_tokens']
            # prompt_lens = samples['prompt_lens']
            # with self.maybe_autocast():
            graph_embeds1, graph_masks1 = self.graph_encoder(graphs1)
            graph_embeds1 = self.ln_graph(graph_embeds1)

            query_tokens1 = self.query_tokens.expand(graph_embeds1.shape[0], -1, -1)
            query_output1 = self.Qformer.bert(
                query_embeds=query_tokens1,
                encoder_hidden_states=graph_embeds1,
                encoder_attention_mask=graph_masks1,
                return_dict=True,
            )
            mol_tokens1 = self.opt_proj(query_output1.last_hidden_state)
            
            graphs2 = samples['graphs2']
            # prompt_lens = samples['prompt_lens']
            # with self.maybe_autocast():
            graph_embeds2, graph_masks2 = self.graph_encoder(graphs2)
            graph_embeds2 = self.ln_graph(graph_embeds2)

            query_tokens2 = self.query_tokens.expand(graph_embeds2.shape[0], -1, -1)
            query_output2 = self.Qformer.bert(
                query_embeds=query_tokens2,
                encoder_hidden_states=graph_embeds2,
                encoder_attention_mask=graph_masks2,
                return_dict=True,
            )
            mol_tokens2 = self.opt_proj(query_output2.last_hidden_state)
            mol_tokens=torch.cat([mol_tokens1,mol_tokens2],dim=1)
            prompt_embeds = self.opt_model.get_input_embeddings()(prompt_tokens.input_ids)

            mol_tokens = mol_tokens.to(prompt_embeds.dtype)

            prompt_embeds[prompt_tokens.is_mol_token] = mol_tokens.flatten(0, 1)

            outputs = self.opt_model.generate(
                inputs_embeds=prompt_embeds,
                attention_mask=prompt_tokens.attention_mask,
                do_sample=do_sample,
                top_p=top_p,
                temperature=temperature,
                num_beams=num_beams,
                max_length=max_length,
                min_length=min_length,
                # pad_token_id=self.pad_token_id,
                eos_token_id=self.eos_token_id,
                repetition_penalty=repetition_penalty,
                length_penalty=length_penalty,
                num_return_sequences=num_captions,
                output_hidden_states=True,
                return_dict_in_generate=True
                # use_cache=False,
            )
            return outputs
            output_text = self.opt_tokenizer.batch_decode(outputs, skip_special_tokens=True)
            
            output_text = [text.strip() for text in output_text]
            return output_text
        else :
            graphs = samples['graphs']
            prompt_tokens = samples['prompt_tokens']
            # prompt_lens = samples['prompt_lens']
            # with self.maybe_autocast():
            graph_embeds, graph_masks = self.graph_encoder(graphs)
            graph_embeds = self.ln_graph(graph_embeds)

            query_tokens = self.query_tokens.expand(graph_embeds.shape[0], -1, -1)
            query_output = self.Qformer.bert(
                query_embeds=query_tokens,
                encoder_hidden_states=graph_embeds,
                encoder_attention_mask=graph_masks,
                return_dict=True,
            )
            mol_tokens = self.opt_proj(query_output.last_hidden_state)
            
            prompt_embeds = self.opt_model.get_input_embeddings()(prompt_tokens.input_ids)
            prompt_embeds[prompt_tokens.is_mol_token] = mol_tokens.flatten(0, 1)

            outputs = self.opt_model.generate(
                inputs_embeds=prompt_embeds,
                attention_mask=prompt_tokens.attention_mask,
                do_sample=do_sample,
                top_p=top_p,
                temperature=temperature,
                num_beams=num_beams,
                max_length=max_length,
                min_length=min_length,
                # pad_token_id=self.pad_token_id,
                eos_token_id=self.eos_token_id,
                repetition_penalty=repetition_penalty,
                length_penalty=length_penalty,
                num_return_sequences=num_captions,
                # use_cache=False,
            )
            output_text = self.opt_tokenizer.batch_decode(outputs, skip_special_tokens=True)
            
            output_text = [text.strip() for text in output_text]
            return output_text

    @torch.no_grad()
    def blip_qa(
        self, 
        samples,
        do_sample=False,
        num_beams=5,
        max_length=128,
        min_length=1,
        top_p=0.9,
        repetition_penalty=1.0,
        length_penalty=1.0,
        num_captions=1,
        temperature=1,
        output_scores=False,
        ):

        device = XXXX-2(self.parameters()).device
        
        ## data processing
        prompts = samples['prompts'] # assume list of strings
        prepared_prompts = []
        mol_list = []
        for p in prompts:
            text, smiles = smiles_handler(p, self.mol_token * self.num_query_token)
            prepared_prompts.append(text)
            mol_list.extend([smiles2data(s) for s in smiles])
        
        prompt_tokens = self.opt_tokenizer(prepared_prompts,
                                           truncation=False,
                                           padding='longest',
                                           add_special_tokens=True,
                                        #    max_length=self.args.max_len[],
                                           return_tensors='pt',
                                           return_attention_mask=True).to(device)
        
        ## forward function
        prompt_embeds = self.opt_model.get_input_embeddings()(prompt_tokens.input_ids)
        
        if len(mol_list) > 0:
            graphs = self.collater(mol_list).to(device)
            is_mol_token = (prompt_tokens.input_ids == self.mol_token) # shape = [B, max_len]
            ## graph forward
            graph_embeds, graph_masks = self.graph_encoder(graphs)
            graph_embeds = self.ln_graph(graph_embeds, graph_masks)
            query_tokens = self.query_tokens.expand(graph_embeds.shape[0], -1, -1)
            query_output = self.Qformer.bert(
                query_embeds=query_tokens,
                encoder_hidden_states=graph_embeds,
                encoder_attention_mask=graph_masks, # fixme: check whether this mask is correct
                return_dict=True,
            )
            mol_tokens = self.opt_proj(query_output.last_hidden_state) # shape = [mol_num, num_query_token, D]
            ## replace mol tokens
            prompt_embeds[is_mol_token] = mol_tokens.flatten(0, 1)
        
        if output_scores:
            outputs = self.opt_model.generate(
                    inputs_embeds=prompt_embeds,
                    attention_mask=prompt_tokens.attention_mask,
                    do_sample=do_sample,
                    top_p=top_p,
                    temperature=temperature,
                    num_beams=num_beams,
                    max_length=max_length,
                    min_length=min_length,
                    # pad_token_id=self.pad_token_id,
                    eos_token_id=self.eos_token_id,
                    repetition_penalty=repetition_penalty,
                    length_penalty=length_penalty,
                    num_return_sequences=num_captions,
                    output_scores=True,
                    return_dict_in_generate=True
                    # use_cache=False,
            )
            return outputs
        else:
            outputs = self.opt_model.generate(
                    inputs_embeds=prompt_embeds,
                    attention_mask=prompt_tokens.attention_mask,
                    do_sample=do_sample,
                    top_p=top_p,
                    temperature=temperature,
                    num_beams=num_beams,
                    max_length=max_length,
                    min_length=min_length,
                    # pad_token_id=self.pad_token_id,
                    eos_token_id=self.eos_token_id,
                    repetition_penalty=repetition_penalty,
                    length_penalty=length_penalty,
                    num_return_sequences=num_captions,
                    # use_cache=False,
                )
            output_text = self.opt_tokenizer.batch_decode(outputs, skip_special_tokens=True)
            output_text = [text.strip() for text in output_text]
            return output_text
    
    @torch.no_grad()
    def opt_qa(
        self, 
        samples,
        do_sample=False,
        num_beams=5,
        max_length=128,
        min_length=1,
        top_p=0.9,
        repetition_penalty=1.0,
        length_penalty=1.0,
        num_captions=1,
        temperature=1,
        output_scores=False,
        ):

        device = (self.parameters()).device
        ## data processing
        prompts = samples['prompts'] # assume list of strings
        prompts = [escape_custom_split_sequence(p) for p in prompts]
        
        prompt_tokens = self.opt_tokenizer(prompts,
                                           truncation=False,
                                           padding='longest',
                                           add_special_tokens=True,
                                        #    max_length=self.args.max_len[],
                                           return_tensors='pt',
                                           return_attention_mask=True).to(device)
        
        prompt_embeds = self.opt_model.get_input_embeddings()(prompt_tokens.input_ids)

        if output_scores:
            ## forward function
            outputs = self.opt_model.generate(
                    inputs_embeds=prompt_embeds,
                    attention_mask=prompt_tokens.attention_mask,
                    do_sample=do_sample,
                    top_p=top_p,
                    temperature=temperature,
                    num_beams=num_beams,
                    max_length=max_length,
                    min_length=min_length,
                    # pad_token_id=self.pad_token_id,
                    eos_token_id=self.eos_token_id,
                    repetition_penalty=repetition_penalty,
                    length_penalty=length_penalty,
                    num_return_sequences=num_captions,
                    # use_cache=False,
                    output_scores=True,
                    return_dict_in_generate=True
                )
            return outputs
        else:
            ## forward function
            outputs = self.opt_model.generate(
                    inputs_embeds=prompt_embeds,
                    attention_mask=prompt_tokens.attention_mask,
                    do_sample=do_sample,
                    top_p=top_p,
                    temperature=temperature,
                    num_beams=num_beams,
                    max_length=max_length,
                    min_length=min_length,
                    # pad_token_id=self.pad_token_id,
                    eos_token_id=self.eos_token_id,
                    repetition_penalty=repetition_penalty,
                    length_penalty=length_penalty,
                    num_return_sequences=num_captions,
                    # use_cache=False,
                )
            output_text = self.opt_tokenizer.batch_decode(outputs, skip_special_tokens=True)
            output_text = [text.strip() for text in output_text]
            return output_text
        
    @torch.no_grad()
    def probe_qformer(
        self, 
        batch,
        do_sample=False,
        num_beams=5,
        max_length=128,
        min_length=1,
        top_p=0.9,
        repetition_penalty=1.0,
        length_penalty=1.0,
        num_captions=1,
        temperature=1,
        ):
        with self.maybe_autocast():
            device = XXXX-2(self.parameters()).device
            
            graphs, smiles_prompt_tokens, texts = batch
            graphs = graphs.to(device)
            ## graph forward
            graph_embeds, graph_masks = self.graph_encoder(graphs)
            graph_embeds = self.ln_graph(graph_embeds, graph_masks)
            query_tokens = self.query_tokens.expand(graph_embeds.shape[0], -1, -1)
            query_output = self.Qformer.bert(
                query_embeds=query_tokens,
                encoder_hidden_states=graph_embeds,
                encoder_attention_mask=graph_masks, # fixme: check whether this mask is correct
                return_dict=True,
            )
            mol_tokens = self.opt_proj(query_output.last_hidden_state) # shape = [mol_num, num_query_token, D]
            B, num_q, D = mol_tokens.shape
            
            ## 
            embed_func = self.opt_model.get_input_embeddings()
            embed_weight = embed_func.weight # shape = [vocab_size, D]
            
            dis_metric = 'cos'
            topk = 10
            if dis_metric == 'cos':
                mol_tokens = F.normalize(mol_tokens, dim=-1, p=2)
                embed_weight = F.normalize(embed_weight, dim=-1, p=2)
                sim = mol_tokens.flatten(0, 1) @ embed_weight.T # shape = [mol_num * num_query_token, vocab_size]
            elif dis_metric == 'euc':
                sim = - torch.cdist(mol_tokens.flatten(0, 1), embed_weight, p=2)
                assert sim.shape == (B * num_q, embed_weight.shape[0])
            else:
                raise NotImplementedError()
            _, topk_ids = torch.topk(sim, k=topk, dim=-1) # shape = [mol_num * num_query_token, k]
            knn_decode_strings = self.opt_tokenizer.batch_decode(topk_ids.flatten())
            knn_decode_strings = np.asarray(knn_decode_strings).reshape(B, num_q, topk).tolist() # shape = [mol_num, num_query_token, topk]
            knn_decode_strings = [[' '.join(ii) for ii in i] for i in knn_decode_strings] # shape = [mol_num, num_query_token]
            if False:
                ### print for presentation
                assert len(knn_decode_strings) == len(texts)
                for predict, text in zip(knn_decode_strings, texts):
                    print('----------------------------')
                    print(predict)
                    print(text)
            return knn_decode_strings
            


<>:524: SyntaxWarning: 'int' object is not callable; perhaps you missed a comma?
<>:693: SyntaxWarning: 'int' object is not callable; perhaps you missed a comma?
<>:524: SyntaxWarning: 'int' object is not callable; perhaps you missed a comma?
<>:693: SyntaxWarning: 'int' object is not callable; perhaps you missed a comma?
/tmp/ipykernel_49365/3830296720.py:524: SyntaxWarning: 'int' object is not callable; perhaps you missed a comma?
  device = XXXX-2(self.parameters()).device
/tmp/ipykernel_49365/3830296720.py:693: SyntaxWarning: 'int' object is not callable; perhaps you missed a comma?
  device = XXXX-2(self.parameters()).device


In [5]:

blip2opt = Blip2OPT(args.bert_name, args.gin_num_layers, args.gin_hidden_dim, args.drop_ratio, 
args.tune_gnn, args.num_query_token, args.cross_attention_freq, args.llm_tune, args.peft_dir, args.opt_model, args.prompt, args)

/work/home/ac15y5ara4/cz/MolTC-main/model/blip2.py:99: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  ckpt = torch.load('/work/home/ac15y5ara4/cz/MolTC-main/gin_pretrained/gr

bert load scibert


/tmp/ipykernel_49365/3830296720.py:524: SyntaxWarning: 'int' object is not callable; perhaps you missed a comma?
  device = XXXX-2(self.parameters()).device
/tmp/ipykernel_49365/3830296720.py:693: SyntaxWarning: 'int' object is not callable; perhaps you missed a comma?
  device = XXXX-2(self.parameters()).device
/tmp/ipykernel_49365/3830296720.py:524: SyntaxWarning: 'int' object is not callable; perhaps you missed a comma?
  device = XXXX-2(self.parameters()).device
/tmp/ipykernel_49365/3830296720.py:693: SyntaxWarning: 'int' object is not callable; perhaps you missed a comma?
  device = XXXX-2(self.parameters()).device


ValueError: The state dictionary of the model you are trying to load is corrupted. Are you sure it was properly saved?

In [30]:
# checkpoint_path = "/home/cz/MolTC-main/all_checkpoints/oridaima/epoch=09.ckpt"
# checkpoint = torch.load(checkpoint_path)
# state_dict = checkpoint['state_dict']


FileNotFoundError: [Errno 2] No such file or directory: '/home/cz/MolTC-main/all_checkpoints/oridaima/epoch=09.ckpt'

In [ ]:
blip2opt.load_state_dict(state_dict,strict=False)

NameError: name 'state_dict' is not defined

In [ ]:
blip2opt

Blip2OPT(
  (graph_encoder): GNN(
    (x_embedding1): Embedding(120, 300)
    (x_embedding2): Embedding(3, 300)
    (gnns): ModuleList(
      (0-4): 5 x GINConv()
    )
    (batch_norms): ModuleList(
      (0-4): 5 x BatchNorm1d(300, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
  )
  (ln_graph): LayerNorm((300,), eps=1e-05, elementwise_affine=True)
  (Qformer): BertLMHeadModel(
    (bert): BertModel(
      (embeddings): BertEmbeddings(
        (word_embeddings): None
        (position_embeddings): None
        (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
        (dropout): Dropout(p=0.1, inplace=False)
      )
      (encoder): BertEncoder(
        (layer): ModuleList(
          (0): BertLayer(
            (attention): BertAttention(
              (self): BertSelfAttention(
                (query): Linear(in_features=768, out_features=768, bias=True)
                (key): Linear(in_features=768, out_features=768, bias=True)
               

In [31]:
from data_provider.stage2_dm import Stage2DM,Stage2DM_double,Stage2DM_double_value,Stage2DM_double_DDIvalue,Stage2DM_double_fgtvalue,Stage2DM_universal
tokenizer = blip2opt.opt_tokenizer
dm = Stage2DM_double(args.mode, args.num_workers, args.batch_size, '/home/cz/MolTC-main/data/ddi_data/Zhangddi_data/train/', args.text_max_len, 
tokenizer, args.zijiegou,args)

In [32]:
train_loader = dm.val_dataloader()[0]

	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [33]:
a = next(iter(train_loader))
a

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


(DataBatch(x=[613, 2], edge_index=[2, 1312], edge_attr=[1312, 2], rdkit_indices=[613], batch=[613], ptr=[21]),
 DataBatch(x=[507, 2], edge_index=[2, 1092], edge_attr=[1092, 2], rdkit_indices=[507], batch=[507], ptr=[21]),
 {'input_ids': tensor([[   2,  243,   21,  ...,    1,    1,    1],
         [   2,  243,   21,  ...,    1,    1,    1],
         [   2,  243,   21,  ...,    1,    1,    1],
         ...,
         [   2,  243,   21,  ...,    1,    1,    1],
         [   2,  243,   21,  ...,   36, 6402,   48],
         [   2,  243,   21,  ...,    1,    1,    1]]), 'token_type_ids': tensor([[0, 0, 0,  ..., 0, 0, 0],
         [0, 0, 0,  ..., 0, 0, 0],
         [0, 0, 0,  ..., 0, 0, 0],
         ...,
         [0, 0, 0,  ..., 0, 0, 0],
         [0, 0, 0,  ..., 0, 0, 0],
         [0, 0, 0,  ..., 0, 0, 0]]), 'attention_mask': tensor([[1, 1, 1,  ..., 0, 0, 0],
         [1, 1, 1,  ..., 0, 0, 0],
         [1, 1, 1,  ..., 0, 0, 0],
         ...,
         [1, 1, 1,  ..., 0, 0, 0],
         [1, 1, 

In [34]:
graphs1,graphs2, prompt_tokens, texts = a
#  graphs1 = graphs1.to(torch.bfloat16)
# graphs2 = graphs2.to(torch.bfloat16)
# prompt_tokens = prompt_tokens.to(torch.bfloat16)
samples = {'graphs1': graphs1, 'graphs2': graphs2,'prompt_tokens': prompt_tokens}

In [35]:
samples["prompt_tokens"]["input_ids"].shape

torch.Size([20, 228])

In [ ]:
prompt_tokens.is_mol_token


tensor([[False, False, False,  ..., False, False, False],
        [False, False, False,  ..., False, False, False],
        [False, False, False,  ..., False, False, False],
        ...,
        [False, False, False,  ..., False, False, False],
        [False, False, False,  ..., False, False, False],
        [False, False, False,  ..., False, False, False]])

In [37]:
args.max_len

30

In [113]:
predictions = blip2opt.generate(
            samples, 
            do_sample=True,
            num_beams=args.num_beams-3,
            max_length=11,
            min_length=args.min_len,
            num_captions=1
        )

开始生成


In [89]:
predictions.hidden_states[-1][5][0].shape

torch.Size([1, 2048])

In [119]:
print(len(predictions.hidden_states))
print(len(predictions.hidden_states[-1]))
# print(len(predictions.hidden_states[-1][0]))
#maxlen-1 * hidden_layers *(batchsize*numbeams*captions)*1*2048
a=predictions.hidden_states[-4][-1][::2].squeeze(1)

10
25


In [127]:
# 模拟的 caption_outputs 数据
caption_outputs = [
    (
        ["prediction_1a", "prediction_1b"],   # list_predictions
        ["target_1a", "target_1b"],           # list_targets
        torch.tensor([1, 0])              # list_mlppredicts
    ),
    (
        ["prediction_2a", "prediction_2b"],
        ["target_2a", "target_2b"],
        torch.tensor([0, 1])
    )
]


In [128]:
# 解包数据
list_predictions, list_targets, list_mlppredicts = zip(*caption_outputs)

# 展开 predictions 和 targets 列表
predictions = [i for ii in list_predictions for i in ii]
targets = [i for ii in list_targets for i in ii]
mlppredicts = [i.item() for ii in list_mlppredicts for i in ii]  # 转为标量列表

# 输出结果
print("Predictions:", predictions)
print("Targets:", targets)
print("MLP Predicts:", mlppredicts)


Predictions: ['prediction_1a', 'prediction_1b', 'prediction_2a', 'prediction_2b']
Targets: ['target_1a', 'target_1b', 'target_2a', 'target_2b']
MLP Predicts: [1, 0, 0, 1]


In [120]:
a.shape

torch.Size([20, 2048])

In [106]:
predictions.sequences

tensor([[    0,   304,   381,  2351,   343,   281,   243,    39,    42,    35,
          4293],
        [    0,   473, 35427,  4545,   425,    35,    62,    40,    68,    39,
            62],
        [    0,    30,    39,    31,   381,  1093,  6655,   343,  3129,   345,
           281],
        [    0,   111,   606,    85,  5145,    85,  6231,   243,    51,   243,
            29],
        [    0,    30,   592,    31,    68,    34,    68,    35, 21882,  3082,
          6246],
        [    0,   325,   851,   243,    51,   243,    30,    24,    63,    24,
           243],
        [    0,    30,    63,  1361,   568,   774,  5570,    31,   221,     1,
             1],
        [    0,   105,   243,    35,  5410,   343,   286,  2456,  2157,   299,
          1199],
        [    0,    36,   732,  5262,   496,   391, 20673,   452,   243,    39,
            40],
        [    0,   243,    44,    38,  2303,   343,  1142,    36,  5987,  1652,
         14837],
        [    0,   325,   851,   243,    

In [105]:
predictions.hidden_states[-4][-1][:].shape

torch.Size([20, 1, 2048])

In [61]:
# 获取生成的序列和隐藏状态
sequences = predictions.sequences  # (batch_size*num_return_sequences, sequence_length)
hidden_states = predictions.hidden_states  # Tuple of shape (sequence_length, num_layers, (batch_size*num_beams*num_return_sequences, generated_length, hidden_size))

# 获取最后一层隐藏状态
# hidden_states[-1] 是最后生成的 token 的隐藏状态
# hidden_states[-1][-1] 是最后一层的隐藏状态
last_hidden_states = hidden_states[-1][-1]  # Shape: (batch_size*num_return_sequences, sequence_length, hidden_size)

# 如果需要获取每个 token 的最后一层隐藏状态
final_hidden_states_per_token = last_hidden_states[:, :sequences.shape[1], :]  # 对应每个生成的 token 的最后一层隐藏状态

print(final_hidden_states_per_token.shape)


torch.Size([100, 1, 2048])
